# "'한국전력 적자' 네이버 6개월 뉴스 크롤링"
> "네이버 뉴스에서 '한국전력 적자'를 검색어로 한 최근 6개월 간 기사 수집"

- toc:true- branch: master
- badges: true
- comments: true
- author: Ryu Han
- categories: [kepco, climate crisis]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/kepco_6months.csv')

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.10-py2.py3-none-any.whl (139 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (108 kB/s)
Reading package lis

In [ ]:
!pip install webdriver_manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully uninstalled urllib3-1.26.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.3.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
len(df)

4000

In [ ]:
df.head(3)

,agency,title,naver_url
0,주간조선언론사 선정,한전 적자에 휘청…정치가 밀어붙인 한전공대의 운명,https://n.news.naver.com/mnews/article/053/000...
1,매경이코노미언론사 선정,김종갑 前 한전 사장이 밝히는 ‘한전 적자’ 해법,https://n.news.naver.com/mnews/article/024/000...
2,조선비즈언론사 선정,"전력 도매가격, 상한제 도입 앞두고 급등 조짐",https://n.news.naver.com/mnews/article/366/000...


In [ ]:
# 네이버 url이 없는 기사는 분석에서 제외한다
df_drop = df.dropna(axis=0)
len(df_drop)

2252

naver_url 없는 행을 삭제하니, 기사가 4000개에서 2252개로 줄음

In [ ]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)


In [ ]:
len(df_drop)

2252

In [ ]:
naver_urls = list()
for i in df_drop.naver_url:
  naver_urls.append(i)

In [ ]:
len(naver_urls)

2252

In [ ]:

###naver 기사 본문 가져오기###

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    # 뉴스 본문 가져오기

    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    contents.append(content)

# print(titles)
# print(contents)

# 데이터프레임으로 정리(titles,url,contents)
import pandas as pd

news_df = pd.DataFrame({'link': naver_urls, 'content': contents})

news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8-sig')

In [ ]:
df = pd.read_csv('/content/NaverNews_한국전력 적자.csv')

In [ ]:
df.head(3)

,link,content
0,https://n.news.naver.com/mnews/article/053/000...,[\n\n\n\n\n지난 3월 2일 전남 나주시 빛가람동에서 열린 한국에너지공과대학...
1,https://n.news.naver.com/mnews/article/024/000...,[\n전기요금 원가 연계 없으면 ‘정상화’ 불가능김종갑 한양대 특훈교수는 2018년...
2,https://n.news.naver.com/mnews/article/366/000...,[\n민간 발전사는 상한제 도입 반대 시위 한국전력(015760)이 발전사들에게 전...


In [ ]:
df = df.rename(columns={'link':'naver_url'})
df.head(3)

,naver_url,content
0,https://n.news.naver.com/mnews/article/053/000...,[\n\n\n\n\n지난 3월 2일 전남 나주시 빛가람동에서 열린 한국에너지공과대학...
1,https://n.news.naver.com/mnews/article/024/000...,[\n전기요금 원가 연계 없으면 ‘정상화’ 불가능김종갑 한양대 특훈교수는 2018년...
2,https://n.news.naver.com/mnews/article/366/000...,[\n민간 발전사는 상한제 도입 반대 시위 한국전력(015760)이 발전사들에게 전...


In [ ]:
df_final = pd.merge(df, df_drop)

In [ ]:
df_final = df_final[['agency', 'naver_url', 'title', 'content']]
df_final.head(3)

,agency,naver_url,title,content
0,주간조선언론사 선정,https://n.news.naver.com/mnews/article/053/000...,한전 적자에 휘청…정치가 밀어붙인 한전공대의 운명,[\n\n\n\n\n지난 3월 2일 전남 나주시 빛가람동에서 열린 한국에너지공과대학...
1,매경이코노미언론사 선정,https://n.news.naver.com/mnews/article/024/000...,김종갑 前 한전 사장이 밝히는 ‘한전 적자’ 해법,[\n전기요금 원가 연계 없으면 ‘정상화’ 불가능김종갑 한양대 특훈교수는 2018년...
2,조선비즈언론사 선정,https://n.news.naver.com/mnews/article/366/000...,"전력 도매가격, 상한제 도입 앞두고 급등 조짐",[\n민간 발전사는 상한제 도입 반대 시위 한국전력(015760)이 발전사들에게 전...


In [ ]:
df_final.to_csv('kepco_deficit_6months.csv')